In [1]:
from src.manager import application_manager

In [2]:
manager = application_manager.ApplicationManager()

In [3]:
manager.apiManager.retrieve_mod("the twilight forrest")

name: the twilight forrest
url:  https://api.modrinth.com/v2/search?query=the+twilight+forrest&limit=20&index=relevance&offset=0&facets=%5B%5B%22project_type+%3A+mod%22%5D%5D
- indices :  [10] 
- scores :  [85.3]
best projects:  ['The Aether II']
redirecting to api:  search_id_tag
url:  https://api.modrinth.com/v2/projects?ids=%5B%22JD2NSu5O%22%5D
redirecting to api:  id_tag
url:  https://www.curseforge.com/api/v1/mods/search?filterText=the+twilight+forrest&gameId=432&sortField=1&pageSize=50&index=0
- indices :  [0] 
- scores :  [96.9]
best projects:  ['The Twilight Forest']
redirecting to api:  search_id_tag
url:  https://api.curseforge.com/v1/mods
redirecting to api:  id_tag


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [1]:
import re

def remove_brackets(text):
    return re.sub(r'\[.*?\]', '', text)

In [2]:
text = "Terralit [Forge/Fabric]"

print(remove_brackets(text))

Terralit 


In [43]:
line = "\t\tThe Aether II  "
comment = "\t\t# This is a comment  "
empty = "\t\t"

filepath = r"C:\Users\Lukas\git\Minecraft\MinecraftModpackManager\data\download_list.txt"
with open(filepath, 'r') as f:
    lines = f.readlines()


In [52]:
def parse_file(lines):
    current_categories = []
    previous_indentation = 0
    previous_line = None
    mods = {}
    intendation_steps = None
    
    def add_mod(name, categories):
        print("add mod:", name)
        mods[name] = categories.copy()
    
    for line in lines:
        line = line.rstrip() # remove trailing whitespace
        
        # TODO: more preciise way to determine indentation (separate tabs from spaces)
        indentation = len(line) - len(line.lstrip())
        line = line.strip()
        
        # ignore comments and emtpy lines
        if line.startswith("#") or line is None or line == "":
            continue
        
        # ignore comment part of not-empty lines and strip again
        line = line.split("#")[0].strip()
        
        # calculate intendation difference to previous line and normalize to 1
        diff = indentation - previous_indentation
        if intendation_steps is None:
            if diff > 0:
                intendation_steps = diff
        else:
            diff //= intendation_steps
            
        print("intendation diff:", diff, "categories:", current_categories)
        if previous_line:
            if diff > 0:
                # previous line was a category
                current_categories.append(previous_line)
                
            elif diff < 0:
                # prvious line was a mod
                add_mod(previous_line, current_categories)
                # remove |diff| number of categories
                current_categories = current_categories[:diff]
                
            elif diff == 0:
                # add mod:
                add_mod(previous_line, current_categories)
                
        previous_line = line
        previous_indentation = indentation
        
    # Process the last line of the file
    if previous_line:
        add_mod(previous_line, current_categories)
        
        
    print(mods)
            

In [53]:
parse_file(lines)

intendation diff: 0 categories: []
intendation diff: 0 categories: []
add mod: ServerSync


AttributeError: 'list' object has no attribute 'deepcopy'